# Search Directory Tutorial

This notebook demonstrates how to use the `SearchDirectory` class to create embeddings and query a set of documents using a FAISS index.

The process can be broken down into the following steps:
1. Get text information from documents in a directory
2. Chunk the text data
3. Load an embedding model
4. Embed the chunked text
5. Create a FAISS index
6. Use a query to search the FAISS index

In [1]:
import os
import shutil
from faiss_search import SearchDirectory

c:\Users\TYWILSON\OneDrive - HC-SC PHAC-ASPC\Documents\GitHub\faiss-search\.venv\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


### 1. Get text information from documents in a directory

This step can be skipped if you already have a CSV containing the document names/file paths and the text data.

In [5]:
# specify a path to save the chunking, embedding, and faiss index to
os.makedirs('sample_search_docs', exist_ok=True)
search_dir_path = "sample_search_docs"

# create a SearchDirectory object
search = SearchDirectory(search_dir_path)

In [6]:
# specify the path with the files to extract text from
resource_path = "tests/resources/sample_text_files"

# generate a CSV report that contains text information
search.report_from_directory(resource_path)

Processing files: 20 files completed [00:00, 50.49 files completed/s]
Processing batches: 1 batches completed [00:00,  2.51 batches completed/s]


### 2. Chunk the text data from the report

You can either pass no arguments and it will use the `report.csv` generated in the previous step or you can specify the file path of another CSV file containing text data along with the column names of the file path and text content.

In [7]:
# generate chunks from the report
search.chunk_text()

Total rows (excluding header): 20


Processing rows: 100%|██████████| 20/20 [00:00<00:00, 9787.20it/s]

Chunking complete and saved to 'data_chunked.csv'.


In [ ]:
# generate chunks from a CSV file
search.chunk_text("tests/resources/search_directory_test_files/report_modified.csv",
                  "path",
                  "content")

### 3. Specify the embedding model to use

In [8]:
search.load_embedding_model("paraphrase-MiniLM-L3-v2")

c:\Users\TYWILSON\OneDrive - HC-SC PHAC-ASPC\Documents\GitHub\faiss-search\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### 4. Perform embeddings on the chunked text data

By default, this will split the task into batches that are saved to store progress during long computations. This is demonstrated by specifying the `batch_size` to be 20 chunks. The embeddings can also be broken down further by specifying the start and end chunks. The function is also designed to not recompute any chunks that have already been saved.

Once all the chunks are computed and saved then they are combined and saved to `embeddings.npy`

In [9]:
search.embed_text(row_start=0, row_end=-1, batch_size=20)

100%|██████████| 20/20 [00:04<00:00,  4.62it/s]


Embedding batch complete and saved to embeddings (0-20).npy').


100%|██████████| 20/20 [00:00<00:00, 38.44it/s]


Embedding batch complete and saved to embeddings (20-40).npy').


100%|██████████| 20/20 [00:00<00:00, 40.23it/s]


Embedding batch complete and saved to embeddings (40-60).npy').


100%|██████████| 16/16 [00:00<00:00, 38.02it/s]

Embedding batch complete and saved to embeddings (60-76).npy').
Embeddings combined and saved to embeddings.npy


### 5. Create the FAISS index

Multiple different types of FAISS indexes can be created with different hyperparameters. The functionality of using and creating FAISS indexes is demonstrated in more depth in `faiss_demo.ipynb`. This class uses the same methods as that demo but will always save the FAISS index after creating them.

In [10]:
search.create_flat_index()

### 6. Query the FAISS index and find the most similar documents

Specify a query and the number of similar chunks to return (as well as any hyperparameters depending on the FAISS index used) and this will return a data frame with the most similar chunks (accoring to the embedding and FAISS models used).

In [11]:
query = "What is the meaning of life, the universe, and everything?"
search.search(query, k=3)

,file_path,content
10,tests\resources\sample_text_files\climate_chan...,The earth's climate is naturally variable on a...
43,tests\resources\sample_text_files\history_of_c...,"The Huron-Wendat of the Great Lakes Region, li..."
56,tests\resources\sample_text_files\origin_of_na...,"Origin of the name ""Canada""\nToday, it seems i..."


Clean up created files

In [12]:
shutil.rmtree("sample_search_docs")